In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 79.2 MB/s eta 0:00:00
  Attempting uninstall: autograd
    Found existing installation: autograd 1.6.2
    Uninstalling autograd-1.6.2:
      Successfully uninstalled autograd-1.6.2


Importing Libraries

In [2]:
import pennylane as qml
import numpy as np
import math

Define Oracle

In [3]:
def oracle(wires, n, num_list):

    #define the Matrix
    matrix=np.identity(2**len(wires))

    #flip the required state
    for i in range(len(num_list)):
        if n==num_list[i]:
            matrix[i][i]=-1
            break

    #make the matrix as gate
    qml.QubitUnitary(matrix, wires=wires)

Define Superpositon state

In [4]:
def equal_superposition(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)

In [5]:
def find_number(num_list, n):

    #define number of qubits
    NUM_QUBITS = math.ceil(math.log2(len(num_list)))

    len_list=len(num_list)
    wires = list(range(NUM_QUBITS))

    #define device
    dev = qml.device("default.qubit", wires=NUM_QUBITS)

    #define Circuit
    @qml.qnode(dev)
    def circuit():
        #number of iteration to perform grovers search
        iterations = int(np.round(np.sqrt(len_list))*np.pi/4)

        #initial state preparation
        equal_superposition(wires)

        #grover's iterator
        for _ in range(iterations):
            oracle(wires, n, num_list)
            qml.templates.GroverOperator(wires)

        #return probabilities
        return qml.probs(wires=wires)

    res=circuit()
    #get the highest probability index and value
    m=np.argmax(res)
    v=np.max(res)

    #if the probability value is not equal to another, return the index else -1
    if v==res[0] and v==res[-1]:
        return -1
    else:
        return m

In [6]:
def find_the_primes_numbers(number_1,number_2):

    #for each item in the list, check the operation, if found return index else return "Not Found"
    for i in range(len(number_2)):
        if find_number(number_2,number_1-number_2[i])>=0:
            return number_2[i],number_1-number_2[i]
    return "Not Found"

In [7]:
find_the_primes_numbers(18,[2,3,5,7,11,13])

(5, 13)

In this case, we are using only log(n) qubits where n is the length of the list. In this case, we only trying to find that number in the list, which is a result of substraction of a the given number and the number we are pointing in the list. Over here, we are using grover search operation for finding the number.